In [ ]:
import torch

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import pandas as pd
import re

# Initialize an empty list to store data for the DataFrame
data = []

# Define the path to your text file
file_path = '/content/drive/MyDrive/Abreviation/CASI/CASI.txt'

# Read the file line by line
with open(file_path, 'r',encoding='utf-8', errors='ignore') as file:
  for line in file:
    # Split the line by the '|' delimiter
    parts = line.split('|')

    # Extract elements based on the expected format
    if len(parts) >= 7:  # Ensure the line has the expected number of fields
        abr = parts[0]
        long_form = parts[1]
        abr_text = parts[2]
        start_abr = int(parts[3])
        end_abr = int(parts[4])
        section = parts[5]  # May be empty if no section is specified
        text = parts[6]

        # Append the extracted fields to the data list as a dictionary
        data.append({
            'abr': abr,
            'long': long_form,
            'abr_text': abr_text,
            'start_abr': start_abr,
            'end_abr': end_abr,
            'section': section,
            'text': text
        })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the resulting DataFrame
print(df)


       abr                                      long abr_text  start_abr  \
0       AB                                  abortion      AB.        231   
1       AB                                  abortion      AB.        249   
2       AB                                  abortion       AB        223   
3       AB                                  abortion      AB.        194   
4       AB                                  abortion       AB        114   
...    ...                                       ...      ...        ...   
37495  VAD  vincristine adriamycin and dexamethasone     VAD.        139   
37496  VAD  vincristine adriamycin and dexamethasone     VAD,        172   
37497  VAD  vincristine adriamycin and dexamethasone      VAD        250   
37498  VAD  vincristine adriamycin and dexamethasone      VAD        181   
37499  VAD  vincristine adriamycin and dexamethasone      VAD         60   

       end_abr                         section  \
0          233                       

In [ ]:
data=pd.DataFrame(df, columns=('abr','long','text'))

In [ ]:
# Define the target abbreviations
df=data
abbreviations = ["AMA", "ASA", "BAL","BK","C3","CVA","CVP","CVS","ER","FISH","NAD","OTC","SBP"]

# Filter rows containing any of the abbreviations
filtered_df = df[df["abr"].str.contains('|'.join(abbreviations), flags=re.IGNORECASE, na=False)]

# Sample 500 rows from the filtered data
#df_mini = filtered_df.sample(n=500, replace=False, random_state=1992)


In [ ]:
# Check the distribution of abbreviations in the sampled data
print(filtered_df['abr'].value_counts())


abr
AMA     500
ASA     500
BAL     500
BK      500
C3      500
CVA     500
CVP     500
CVS     500
ER      500
FISH    500
NAD     500
OTC     500
SBP     500
Name: count, dtype: int64


In [ ]:
# Check the distribution of abbreviations in the sampled data
data=filtered_df
print(data['abr'].value_counts())
print(data.shape)

abr
AMA     500
ASA     500
BAL     500
BK      500
C3      500
CVA     500
CVP     500
CVS     500
ER      500
FISH    500
NAD     500
OTC     500
SBP     500
Name: count, dtype: int64
(6500, 3)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
# Tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
def split_into_chunks(text, tokenizer, chunk_size=512):
    # Ensure the input is a string, not a list
    if isinstance(text, list):
        text = " ".join(text)  # Join the list elements into a single string

    tokens = tokenizer.tokenize(text)  # Tokenize the text
    return [" ".join(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]

class AbbreviationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, chunk_size=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.chunk_size = chunk_size

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        input_text = f"{row['abr']} - {row['text']}"
        target_text = row["long"]

        # Split the input text into chunks
        chunks = split_into_chunks(input_text, self.tokenizer, chunk_size=self.chunk_size)

        # Tokenize input chunks and target text
        inputs = self.tokenizer(
            " ".join(chunks),  # Join the chunks back into a string
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0).to(device),
            "attention_mask": inputs["attention_mask"].squeeze(0).to(device),
            "labels": targets["input_ids"].squeeze(0).to(device),
        }
train_df = data.sample(frac=0.80, random_state=42)
test_df= data.drop(train_df.index)
print(train_df.shape)
print(test_df.shape)
train_dataset = AbbreviationDataset(train_df, tokenizer)
test_dataset = AbbreviationDataset(test_df, tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

(30000, 3)
(7500, 3)


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Load model and move to device
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 5

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
    model.save_pretrained("/content/drive/MyDrive/Abreviation/bart")
    tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart")
    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/5, Training Loss: 1.7524


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/5, train Loss: 0.0348
train Metrics: Accuracy: 0.5742, Precision: 0.5742, Recall: 0.5742, F1: 0.5742
test results


...

Epoch 4/5, Validation Loss: 0.0027
Validation Metrics: Accuracy: 0.7315, Precision: 0.7315, Recall: 0.7315, F1: 0.7315
4
Epoch 5/5, Training Loss: 0.0038
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, train Loss: 0.0018
train Metrics: Accuracy: 0.7402, Precision: 0.7402, Recall: 0.7402, F1: 0.7402
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, Validation Loss: 0.0019
Validation Metrics: Accuracy: 0.7285, Precision: 0.7285, Recall: 0.7285, F1: 0.7285


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0072
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0012
train Metrics: Accuracy: 0.7915, Precision: 0.7915, Recall: 0.7915, F1: 0.7915
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0014
Validation Metrics: Accuracy: 0.7800, Precision: 0.7800, Recall: 0.7800, F1: 0.7800


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0013
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0005
train Metrics: Accuracy: 0.8871, Precision: 0.8871, Recall: 0.8871, F1: 0.8871
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0008
Validation Metrics: Accuracy: 0.8685, Precision: 0.8685, Recall: 0.8685, F1: 0.8685


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0008
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0002
train Metrics: Accuracy: 0.9279, Precision: 0.9279, Recall: 0.9279, F1: 0.9279
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9200, Precision: 0.9200, Recall: 0.9200, F1: 0.9200


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0005
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9513, Precision: 0.9513, Recall: 0.9513, F1: 0.9513
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9331, Precision: 0.9331, Recall: 0.9331, F1: 0.9331


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0004
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9475, Precision: 0.9475, Recall: 0.9475, F1: 0.9475
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9269, Precision: 0.9269, Recall: 0.9269, F1: 0.9269


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0002
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9608, Precision: 0.9608, Recall: 0.9608, F1: 0.9608
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0002
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9558, Precision: 0.9558, Recall: 0.9558, F1: 0.9558
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0004
Validation Metrics: Accuracy: 0.9385, Precision: 0.9385, Recall: 0.9385, F1: 0.9385


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0002
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9592, Precision: 0.9592, Recall: 0.9592, F1: 0.9592
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9400, Precision: 0.9400, Recall: 0.9400, F1: 0.9400


In [ ]:
#weighted results
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1

model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

print('test results')
model.eval()
val_loss = 0
val_predictions = []
val_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
# Compute validation metrics
metrics = compute_metrics(val_predictions, val_labels)
print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9592, Precision: 0.9835, Recall: 0.9592, F1: 0.9667
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9400, Precision: 0.9582, Recall: 0.9400, F1: 0.9459


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0002
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9698, Precision: 0.9698, Recall: 0.9698, F1: 0.9698
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9508, Precision: 0.9508, Recall: 0.9508, F1: 0.9508


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9735, Precision: 0.9735, Recall: 0.9735, F1: 0.9735
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9600, Precision: 0.9600, Recall: 0.9600, F1: 0.9600


In [ ]:
#weighted results
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1

model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])
print(f" train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

print('test results')
model.eval()
val_loss = 0
val_predictions = []
val_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])


 train Loss: 0.0000
train Metrics: Accuracy: 0.9735, Precision: 0.9867, Recall: 0.9735, F1: 0.9769
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(f" Validation Loss: {val_loss / len(test_loader):.4f}")
# Compute validation metrics
metrics = compute_metrics(val_predictions, val_labels)
print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


 Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9600, Precision: 0.9697, Recall: 0.9600, F1: 0.9616


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9769, Precision: 0.9769, Recall: 0.9769, F1: 0.9769
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0004
Validation Metrics: Accuracy: 0.9477, Precision: 0.9477, Recall: 0.9477, F1: 0.9477


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9748, Precision: 0.9748, Recall: 0.9748, F1: 0.9748
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9569, Precision: 0.9569, Recall: 0.9569, F1: 0.9569


In [ ]:
#weighted results
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1

model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

print('test results')
model.eval()
val_loss = 0
val_predictions = []
val_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
# Compute validation metrics
metrics = compute_metrics(val_predictions, val_labels)
print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9748, Precision: 0.9890, Recall: 0.9748, F1: 0.9802
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9569, Precision: 0.9743, Recall: 0.9569, F1: 0.9636


In [ ]:
  model.save_pretrained("/content/drive/MyDrive/Abreviation/bartf")
  tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bartf")

In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9727, Precision: 0.9727, Recall: 0.9727, F1: 0.9727
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9531, Precision: 0.9531, Recall: 0.9531, F1: 0.9531


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9871, Precision: 0.9871, Recall: 0.9871, F1: 0.9871
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0004
Validation Metrics: Accuracy: 0.9608, Precision: 0.9608, Recall: 0.9608, F1: 0.9608


In [ ]:
#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs = 1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='micro', zero_division=0)
    recall = recall_score(labels, preds, average='micro', zero_division=0)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

    print('train results')
    model.eval()
    train_loss = 0
    train_predictions = []
    train_labels = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            train_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
    # Compute train metrics
    metrics = compute_metrics(train_predictions, train_labels)
    print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

    print('test results')
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0001
train results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9890, Precision: 0.9890, Recall: 0.9890, F1: 0.9890
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9654, Precision: 0.9654, Recall: 0.9654, F1: 0.9654


In [ ]:
#weighted results
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epoch= 0

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1

model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")

print('test results')
model.eval()
val_loss = 0
val_predictions = []
val_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_loader):.4f}")
# Compute validation metrics
metrics = compute_metrics(val_predictions, val_labels)
print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0000
train Metrics: Accuracy: 0.9890, Precision: 0.9917, Recall: 0.9890, F1: 0.9893
test results


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0003
Validation Metrics: Accuracy: 0.9654, Precision: 0.9688, Recall: 0.9654, F1: 0.9640


In [ ]:
  model.save_pretrained("/content/drive/MyDrive/Abreviation/bartf")
  tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bartf")

('/content/drive/MyDrive/Abreviation/bartf/tokenizer_config.json',
 '/content/drive/MyDrive/Abreviation/bartf/special_tokens_map.json',
 '/content/drive/MyDrive/Abreviation/bartf/vocab.json',
 '/content/drive/MyDrive/Abreviation/bartf/merges.txt',
 '/content/drive/MyDrive/Abreviation/bartf/added_tokens.json')

#End13_80